In [534]:
import pandas as pd
import re
import numpy as np
import math
#from numba import jit

In [535]:
# Settings

f_3a4='backlog3a4-pcba_allocation.csv'
f_supply='test_SCR+OH+Intransit_0924.xlsx'
sheet_scr='scr'
sheet_transit='in-transit'
sheet_oh='OH & transit-time'

ranking_col=['priority_rank', 'ossd_offset', 'fcd_offset','rev_non_rev_rank','C_UNSTAGED_QTY', 'SO_SS','PO_NUMBER']

# backlog offset by transit pad will not consider ocean ship - assuming ocean is to cocver fcst demand but not backlog demand
transit_time={'FOL':{'FOC':2,'other':7}, 
            'JPE':{'JPE':1,'other':7}
           }

pcba_site='FOL'



In [536]:
def change_supply_to_versionless_and_addup_supply(df,org_col='planningOrg',pn_col='TAN'):
    """
    Change PN in supply or OH df into versionless. Add up the qty into the versionless PN.
    :param df: the supply df or oh df
    :param pn_col: name of the PN col. In Cm supply file it's PN, in Kinaxis file it's TAN.
    :return:
    """
    
    regex = re.compile(r'\d{2,3}-\d{4,7}')
    
    df.reset_index(inplace=True)

    # convert to versionless and add temp col
    df.loc[:,pn_col] = df[pn_col].map(lambda x: regex.search(x).group())
    df.loc[:,'org_pn']=df[org_col] + '_' + df[pn_col]

    # add up the duplicate PN (due to multiple versions)
    df.sort_values(by=['org_pn'],inplace=True)
    dup_pn = df[df.duplicated(['org_pn'])]['org_pn'].unique()
    df_sum = pd.DataFrame(columns=df.columns)

    df_sum.set_index([org_col,pn_col,'org_pn'], inplace=True)
    df.set_index([org_col,pn_col,'org_pn'], inplace=True)
        
    for org_pn in dup_pn:
        # print(df_supply[df_supply.PN==pn].sum(axis=1).sum())
        df_sum.loc[(org_pn[:3],org_pn[4:],org_pn), :] = df.loc[(org_pn[:3],org_pn[4:],org_pn), :].sum(axis=0)
    
    df.reset_index(inplace=True)
    df.set_index('org_pn',inplace=True)
    df.drop(dup_pn, axis=0, inplace=True)
    df.reset_index(inplace=True)
    df.set_index([org_col,pn_col,'org_pn'], inplace=True)
    #print(df.columns)
    #df.drop(['level_0','index'],axis=1,inplace=True)
    df = pd.concat([df, df_sum])
    df.reset_index(inplace=True)
    df.drop(['org_pn'],axis=1,inplace=True)
    df.set_index([org_col,pn_col], inplace=True)

    return df

In [537]:

def update_date_with_transit_pad(x,y,transit_time,pcba_site):
    """
    offset transit time to a given date column
    """
    if x in transit_time[pcba_site].keys():
        return y - pd.Timedelta(days=transit_time[pcba_site][x])
    else:
        return y - pd.Timedelta(days=transit_time[pcba_site]['other'])



In [538]:
def generate_df_order_bom_from_flb_tan_col(df_3a4,pcba):
    """
    Generate the BOM usage file from the FLB_TAN col
    :param df_3a4:
    :return:
    """
    regex_pn = re.compile(r'\d{2,3}-\d{4,7}')
    regex_usage = re.compile(r'\([0-9.]+\)')

    df_flb_tan = df_3a4[df_3a4.FLB_TAN.notnull()][['PO_NUMBER','PRODUCT_ID','ORDERED_QUANTITY','FLB_TAN']].copy()
    #df_flb_tan.drop_duplicates(['PRODUCT_ID'], keep='first', inplace=True)

    po_list=[]
    pn_list = []
    usage_list = []
    for row in df_flb_tan.itertuples(index=False):
        po=row.PO_NUMBER
        flb_tan = row.FLB_TAN
        #order_qty = row.ORDERED_QUANTITY
        flb_tan=flb_tan.split('|')
        
        for item in flb_tan:
            try:
                pn = regex_pn.search(item).group()
                usage = regex_usage.search(item).group()
                usage = float(usage[1:-1])
                
                if pn in pcba:
                    po_list.append(po)
                    pn_list.append(pn)
                    usage_list.append(usage)
                    
            except:
                pass
                #print(po_list)
        
    #print(po_list)
    df_order_bom_from_flb = pd.DataFrame({'PO_NUMBER': po_list, 'BOM_PN': pn_list, 'BOM_PN_QTY': usage_list})

    return df_order_bom_from_flb

In [539]:
def update_order_bom_to_3a4(df_3a4, df_order_bom):
    """
    Add PN into 3a4 based on BOM
    :param df_3a4:
    :param df_bom:
    :return: df_3a4, df_missing_bom_pid
    """
    # add the BOM PN through merge method
    df_3a4 = pd.merge(df_3a4, df_order_bom, left_on='PO_NUMBER', right_on='PO_NUMBER', how='left')

    """
    # PID missing BOM data
    missing_bom_pid = df_3a4[df_3a4.TAN.notnull() & df_3a4.PN.isnull()].PRODUCT_ID.unique()
    df_missing_bom_pid = pd.DataFrame({'Missing BOM PID': missing_bom_pid})

    # 对于BOM missing 的采用3a4中已有的TAN
    df_3a4.loc[:, 'PN'] = np.where(df_3a4.TAN.notnull() & df_3a4.PN.isnull(),
                                   df_3a4.TAN,
                                   df_3a4.PN)
    """
    # correct the quantity by multiplying BOM Qty
    df_3a4.loc[:, 'C_UNSTAGED_QTY']=df_3a4.C_UNSTAGED_QTY * (df_3a4.BOM_PN_QTY/df_3a4.ORDERED_QUANTITY)
    df_3a4.loc[:, 'ORDERED_QUANTITY'] = df_3a4.BOM_PN_QTY

    # add indicator for distinct PO filtering
    df_3a4.loc[:,'distinct_po_filter']=np.where(~df_3a4.duplicated('PO_NUMBER'),
                                              'YES',
                                                '')


    return df_3a4

In [540]:
def created_supply_dict_per_scr(df_scr):
    """
    create supply dict based on scr
    supply_dic_tan={'800-42373':[{'2/10':25},{'2/12':4},{'2/15':10},{'2/22':20},{'3/1':10},{'3/5':15}],
               '800-42925':[{'2/12':4},{'2/13':3},{'2/15':12},{'2/23':25},{'3/1':8},{'3/6':10}]}
    """
    supply_dic_tan={}
    
    
    for tan in df_scr.index:
        date_qty_list=[]
        for date in df_scr.columns:
            date_qty={date:df_scr.loc[tan,date]}
            if not math.isnan(df_scr.loc[tan,date]): # 判断数值是否为空
                if df_scr.loc[tan,date]>0: # 不取0值
                    date_qty_list.append(date_qty)
        supply_dic_tan[tan]=date_qty_list
        
    """ Below version spends much logner time to create the dict. use above instead with index simplified.
    for org_tan in df_scr.index:
        org=org_tan[0]
        tan=org_tan[1]
        date_qty_list=[]
        for date in df_scr.columns:
            date_qty={date:df_scr.loc[(org,tan),date]}
            if not math.isnan(df_scr.loc[(org,tan),date]): # 判断数值是否为空
                if df_scr.loc[(org,tan),date]>0: # 不取0值
                    date_qty_list.append(date_qty)
    """
        
    
    return supply_dic_tan

In [541]:
def created_oh_dict_per_df_oh(df_oh,pcba_site):
    """
    create OH dict based on DF OH (excluding PCBA site and only consider OH>0 items)
    supply_dic_tan={(FOC,'800-42373'):25,(FJZ,'800-42925'):100}
    """
    df_oh=df_oh[(df_oh.OH>0)]
    df_oh.reset_index(inplace=True)
    oh_dic_tan={}
    for row in df_oh[1:].itertuples(index=False):
        org=row.planningOrg
        tan=row.TAN
        oh=row.OH
        
        if org!=pcba_site:
            oh_dic_tan[(org,tan)]=oh
    
    return oh_dic_tan


In [542]:
def create_blg_dict_per_sorted_3a4_and_selected_tan(df_3a4,tan):
    """
    create backlog dict for selected tan list from the sorted 3a4 df (considered order prioity and rank)
    """
    blg_dic_tan={}
    for pn in tan:
        dfm=df_3a4[df_3a4.BOM_PN==pn]
        org_qty_po=[]
        for org,qty,po in zip(dfm.ORGANIZATION_CODE,dfm.BOM_PN_QTY,dfm.PO_NUMBER):
            if qty>0:
                org_qty_po.append({org:(qty,po)})

        blg_dic_tan[pn]=org_qty_po
    
    return blg_dic_tan


In [543]:
def allocate_supply_per_supply_and_blg_dic(supply_dic_tan,blg_dic_tan):
    """
    allocate supply based on supply dict and backlog dict
    supply dict is aranged in date order; backlog dict is aranged based on priority to fulfill
    
    examples: 
        blg_dic_tan={'800-42373-01': [{'FJZ': (5, '110077267-1')},{'FJZ': (23, '110011089-4')},...]}
        supply_dic_tan={'800-42373-01':[{'2/10':25},{'2/12':4},{'2/15':10},{'2/22':20},{'3/1':10},{'3/5':15}],
                             '800-42925-01':[{'2/12':4},{'2/13':3},{'2/15':12},{'2/23':25},{'3/1':8},{'3/6':10}]}
    """
    supply_dic_tan_allocated={}
    
    for tan in supply_dic_tan.keys():
        supply_list_tan=supply_dic_tan[tan]  #每一个tan对应的supply list

        if tan in blg_dic_tan.keys(): #
            blg_list_tan=blg_dic_tan[tan]

            # 对supply list中每一个值进行分配给一个或多个订单
            for date_qty in supply_list_tan:
                #print(date_qty)
                supply_date=list(date_qty.keys())[0]
                supply_qty=list(date_qty.values())[0]
                allocation=[] #每一个supply的分配结果

                allocated_po=[] #已经分配给对应数量的po
                # 对每一个po进行数量分配
                for po in blg_list_tan:
                    #print(po)
                    po_qty=list(po.values())[0][0]
                    po_org=list(po.keys())[0]
                    po_number=list(po.values())[0][1]
                    
                    #print(po_qty)
                    #print(supply_qty)
                    if po_qty<supply_qty: #po数量小于supply数量：po被全额满足；supply数量被减掉；已分配的po被记录 （后面跳转到下一个po）
                        allocation.append((po_org,po_qty))
                        supply_qty=supply_qty-po_qty
                        allocated_po.append(po)
                    elif po_qty==supply_qty: #po数量等于supply数量：po被全额满足；已分配的po被记录；跳出本次po循环(进到下一个supply循环)
                        allocation.append((po_org,po_qty))
                        allocated_po.append(po)
                        break
                    else: #po数量大于supply数量：po被部分（=supply qty）满足；po数量被改小；跳出本次po循环(进到下一个supply循环)
                        allocation.append((po_org,supply_qty))
                        new_po_qty=po_qty-supply_qty
                        ind=blg_list_tan.index(po)
                        blg_list_tan[ind]={po_org:(new_po_qty,po_number)}
                        break

                #print(allocated_po)
                # 把已经被分配的po从列表中删除
                for po in allocated_po:
                    #print(po)
                    blg_list_tan.remove(po)   # double check this one of removing PO whether correct or not
                    blg_dic_tan[tan]=blg_list_tan


                # 把supply列表中对应的supply改变成分配的结果
                ind=supply_list_tan.index(date_qty)
                supply_date=list(date_qty.keys())[0]
                supply_qty=list(date_qty.values())[0]
                supply_list_tan[ind]={supply_date:(supply_qty,allocation)}

        #生成新的allocated supply dict
        supply_dic_tan_allocated[tan]=supply_list_tan
    
    return supply_dic_tan_allocated



In [544]:
def aggregate_allocation_for_each_date(a,date_supply_agg):
    """
    # 根据日期及org汇合每个日期dict下的数量 
    a={'2/12': (4, [('FCZ', 1), ('FJZ', 1), ('FJZ', 1), ('FJZ', 1)])}
    date_supply_agg={}
    此函数在aggregate_supply_dic_tan_allocated中引用
    """
    date=list(a.keys())[0]
    supply=list(a.values())[0][1]  
    supply_total_qty=list(a.values())[0][0]  
    
    orgs=[]
    for org_supply in supply:
        if org_supply[0] not in orgs:
            orgs.append(org_supply[0])
    
    allocation_agg=[]
    for org in orgs:
        qty=0
        for org_supply in supply:
            if org_supply[0]==org:
                qty+=org_supply[1]
        
        allocation_agg.append((org,qty))
    
    date_supply_agg={}
    date_supply_agg[date]=(supply_total_qty,allocation_agg)
    
    return date_supply_agg
 

In [545]:
def aggregate_supply_dic_tan_allocated(supply_dic_tan_allocated):
    """
    针对每一个tan按照每一个日期将分配的数量按照org汇总(引用函数aggregate_allocation_for_each_date)
    """
    supply_dic_tan_allocated_agg={}
    for tan,tan_supply in supply_dic_tan_allocated.items():
        tan_supply_list=[]
        for date_supply in tan_supply:
            date_supply_agg={}
            date_supply_agg=aggregate_allocation_for_each_date(date_supply,date_supply_agg)
            tan_supply_list.append(date_supply_agg)

        supply_dic_tan_allocated_agg[tan]=tan_supply_list
    
    return supply_dic_tan_allocated_agg


In [546]:
def fulfill_backlog_by_oh(oh_dic_tan, blg_dic_tan):
    """
    Fulfill the backlog per DF site based on the DF site OH; deduct the backlog qty accordingly.
    examples:
        blg_dic_tan={'800-42373': [{'FJZ': (5, ('110077267-1','2020-4-1'))},{'FJZ': (23, ('110011089-4','2020-4-4'))},...]}
        oh_dic_tan={('FJZ',800-42373'):25,('FCZ',800-42925'):10}
    return: blg_dic_tan
    """
    for org_tan,qty in oh_dic_tan.items():
        oh_org=org_tan[0]
        oh_tan=org_tan[1]
        oh_qty=qty
        
        if oh_tan in blg_dic_tan.keys():  # blg_dic_tan只包含scr中的tan，oh_tan可能不在其中，如不在，不予考虑
            blg_dic_tan_list=blg_dic_tan[oh_tan] #对应tan下的内容
            blg_dic_tan_list_copy=blg_dic_tan_list.copy()
            
            # 按顺序对每一个po进行数量分配
            for org_po in blg_dic_tan_list:
                po_org=list(org_po.keys())[0]
                po_qty = list(org_po.values())[0][0]
                po_number = list(org_po.values())[0][1]
                
                if po_org==oh_org:
                    po_qty_new=po_qty-oh_qty
                    oh_qty_new=oh_qty - po_qty
                    if po_qty_new<=0: #po已被oh cover完，移除po
                        blg_dic_tan_list_copy.remove(org_po)
                        oh_qty=oh_qty_new
                    else: # oh consumed
                        index=blg_dic_tan_list_copy.index(org_po)
                        blg_dic_tan_list_copy[index]={po_org:(po_qty_new,po_number)}
                    
                        break
            # 更新blg_dic_tan
            blg_dic_tan[oh_tan]=blg_dic_tan_list_copy
                
    return blg_dic_tan

In [547]:
def add_allocation_to_scr(df_scr,df_3a4,supply_dic_tan_allocated_agg,pcba_site):
    """
    Add up the allocation results to scr and create the final output file
    """
    pcba_site_temp='A-' + pcba_site
    df_scr.loc[:,'ORG']=pcba_site_temp
    df_scr.reset_index(inplace=True)
    df_scr.set_index(['TAN','ORG'],inplace=True)
    
    # Add in orgs based on 3a4
    df_3a4_p=df_3a4.pivot_table(index=['BOM_PN','ORGANIZATION_CODE'],values='PO_NUMBER',aggfunc=len)
    df_3a4_p.reset_index(inplace=True)
    
    for row in df_3a4_p.itertuples():
        tan=row.BOM_PN
        org=row.ORGANIZATION_CODE
        
        if tan in df_scr.index:
            df_scr.loc[(tan,org),'count']=row.PO_NUMBER
    df_scr.drop('count',axis=1,inplace=True)
    
    # add in allocated qty
    for tan in supply_dic_tan_allocated_agg.keys():
        for date_supply in supply_dic_tan_allocated_agg[tan]:
            date=list(date_supply.keys())[0]
            org_qty=list(date_supply.values())[0][1]
            for x in org_qty:
                df_scr.loc[(tan,x[0]),date]=x[1]

    df_scr.reset_index(inplace=True)
    df_scr.sort_values(by=['TAN','ORG'],ascending=True,inplace=True)
    df_scr.loc[:,'ORG']=df_scr.ORG.map(lambda x: pcba_site if 'A-' in x else x)
    df_scr.set_index(['TAN','ORG'],inplace=True)
    
    return df_scr


In [548]:
def extract_bu_from_scr(df_scr):
    """
    Versionless the PN and extract the BU info from original scr before pivoting
    """
    regex_pn = re.compile(r'\d{2,3}-\d{4,7}')
    
    tan_bu={}
    for row in df_scr.itertuples(index=False):
        tan = regex_pn.search(row.TAN).group()
        tan_bu[tan]=row.BU
    
    return tan_bu


    
    

In [549]:
def process_final_allocated_output(df_scr,tan_bu,df_3a4,df_oh,pcba_site):
    """
    Add back the BU, backlog,oh, intransit info into the final SCR with allocation result; and add the related columns based on calculations.
    """
    df_scr.reset_index(inplace=True)
    
    # add BU info
    df_scr.loc[:,'BU']=df_scr.TAN.map(lambda x: tan_bu[x])
    
    # add backlog qty
    df_3a4_p=df_3a4.pivot_table(index=['ORGANIZATION_CODE','BOM_PN'],values='BOM_PN_QTY',aggfunc=sum)
    df_3a4_p.columns=['Backlog']
    df_3a4_p.reset_index(inplace=True)
    df_scr=pd.merge(df_scr,df_3a4_p,left_on=['ORG','TAN'],right_on=['ORGANIZATION_CODE','BOM_PN'],how='left')
    
    # add df OH
    df_oh.columns=['OH']
    df_oh.reset_index(inplace=True)
    df_oh=df_oh[df_oh.planningOrg!=pcba_site]
    df_scr=pd.merge(df_scr,df_oh,left_on=['ORG','TAN'],right_on=['planningOrg','TAN'],how='left')
    
    # drop the unneeded columns introduced by merge
    df_scr.drop(['ORGANIZATION_CODE','BOM_PN','planningOrg'],axis=1,inplace=True)
    #df_scr.rename(columns={'TAN_x':'TAN'},inplace=True)
    
    # ADD THE gap col and recovery date
    df_scr.loc[:,'Gap_before']=np.where(df_scr.ORG!=pcba_site,
                                         np.where(df_scr.OH.isnull(),
                                                 0 - df_scr.Backlog,
                                                 df_scr.OH - df_scr.Backlog),
                                        None)
    df_scr.loc[:,'Allocation']=np.where(df_scr.ORG!=pcba_site,
                                         df_scr.iloc[:,3:-4].sum(axis=1),
                                        None)
                                                     
    df_scr.loc[:,'Gap_after']=np.where(df_scr.ORG!=pcba_site,
                                         df_scr.Gap_before+df_scr.Allocation,
                                        None)
    
    df_scr.loc[:,'Recovery']=np.where(df_scr.Gap_before>=0,
                                           "No gap",
                                          np.where(df_scr.Gap_after<0,
                                                  'No recovery',
                                                  'TBD'))
    
    # update with the correct recovery date for TBD
    df_scr.set_index(['TAN','ORG','BU','Backlog','OH','Gap_before','Allocation','Gap_after','Recovery'],inplace=True)
    df_scr.reset_index(inplace=True)
    dfx=df_scr[(df_scr.Recovery=='TBD')&(df_scr.ORG!=pcba_site)]
    dfx.set_index(['TAN','ORG'],inplace=True)
    df_scr.set_index(['TAN','ORG'],inplace=True)
    
    for ind in dfx.index:
        dfy=dfx.loc[ind,:]
        dfy=dfy[dfy.notnull()]
            
        last_allocation_date=dfy.index[-1]
        df_scr.loc[ind,'Recovery']=last_allocation_date
        
        
    
    
    df_scr.reset_index(inplace=True)
    df_scr.set_index(['TAN','ORG','BU','Backlog','OH','Gap_before','Allocation','Gap_after','Recovery'],inplace=True)
    
    return df_scr
    

In [550]:
def ss_ranking_overall_new(df_3a4,ranking_col, order_col='SO_SS', new_col='ss_overall_rank'):
    """
    根据priority_cat,OSSD,FCD, REVENUE_NON_REVENUE,C_UNSTAGED_QTY,按照ranking_col的顺序对SS进行排序。最后放MFG_HOLD订单.
    :param df_3a4:
    :param ranking_col:e.g. ['priority_rank', 'ORIGINAL_FCD_NBD_DATE', 'CURRENT_FCD_NBD_DATE','rev_non_rev_rank',
                        'C_UNSTAGED_QTY', 'SO_SS','PO_NUMBER']
    :param order_col:'SO_SS'
    :param new_col:'ss_overall_rank'
    :return: df_3a4
    """
    # Below create a rev_rank for reference -  currently not used in overall ranking
    ### change non-rev orders unstaged $ to 0
    df_3a4.loc[:,'C_UNSTAGED_DOLLARS']=np.where(df_3a4.REVENUE_NON_REVENUE == 'NO',
                                                0,
                                                df_3a4.C_UNSTAGED_DOLLARS)

    #### 生成ss_unstg_rev并据此排序
    # 计算ss_unstg_rev
    ss_unstg_rev = {}
    df_rev = df_3a4.pivot_table(index='SO_SS', values='C_UNSTAGED_DOLLARS', aggfunc=sum)
    for ss, rev in zip(df_rev.index, df_rev.values):
        ss_unstg_rev[ss] = rev[0]
    df_3a4.loc[:, 'ss_unstg_rev'] = df_3a4.SO_SS.map(lambda x: ss_unstg_rev[x])

    """
    # 计算po_rev_unit - non revenue change to 0
    df_3a4.loc[:, 'po_rev_unit'] = np.where(df_3a4.REVENUE_NON_REVENUE == 'YES',
                                            df_3a4.SOL_REVENUE / df_3a4.ORDERED_QUANTITY,
                                            0)

    # 计算ss_rev_unit: 通过po_rev_unit汇总
    ss_rev_unit = {}
    dfx_rev = df_3a4.pivot_table(index='SO_SS', values='po_rev_unit', aggfunc=sum)
    for ss, rev in zip(dfx_rev.index, dfx_rev.values):
        ss_rev_unit[ss] = rev[0]
    df_3a4.loc[:, 'ss_rev_unit'] = df_3a4.SO_SS.map(lambda x: int(ss_rev_unit[x]))
    """

    # create rank#
    rank = {}
    order_list = df_3a4.sort_values(by='ss_unstg_rev', ascending=False).SO_SS.unique()
    for order, rk in zip(order_list, range(1, len(order_list) + 1)):
        rank[order] = rk
    df_3a4.loc[:, 'ss_rev_rank'] = df_3a4.SO_SS.map(lambda x: rank[x])

    # below creates overall ranking col
    ### Step1: 重新定义priority order及排序
    df_3a4.loc[:, 'priority_cat'] = np.where(df_3a4.SECONDARY_PRIORITY.isin(['PR1', 'PR2', 'PR3']),
                                             df_3a4.SECONDARY_PRIORITY,
                                             np.where(df_3a4.FINAL_ACTION_SUMMARY == 'TOP 100',
                                                      'TOP 100',
                                                      np.where(
                                                          df_3a4.FINAL_ACTION_SUMMARY == 'LEVEL 4 ESCALATION PRESENT',
                                                          'L4',
                                                          np.where(df_3a4.BUP_RANK.notnull(),
                                                                   'BUP',
                                                                   None)
                                                          )
                                                      )
                                             )
    #### Update below DO/DX orders to PR1 due to current PR1/2/3 not updated when order change to DPAS from others
    df_3a4.loc[:, 'priority_cat']=np.where((df_3a4.DPAS_RATING.isin(['DO','DX','TAA-DO','TAA-DX']))&(df_3a4.priority_cat.isnull()),
                                           'PR1',
                                           df_3a4.priority_cat)
    #### Give them a rank
    df_3a4.loc[:, 'priority_rank'] = np.where(df_3a4.priority_cat=='PR1',
                                            1,
                                            np.where(df_3a4.priority_cat =='PR2',
                                                     2,
                                                     np.where(df_3a4.priority_cat =='PR3',
                                                              3,
                                                              np.where(df_3a4.priority_cat == 'TOP 100',
                                                                        4,
                                                                        np.where(df_3a4.priority_cat == 'L4',
                                                                                5,
                                                                                np.where(df_3a4.priority_cat=='BUP',
                                                                                         6,
                                                                                         None)
                                                                                )
                                                                        )
                                                                )
                                                     )
                                              )

    ##### Step2: Give revenue/non-revenue a rank
    df_3a4.loc[:,'rev_non_rev_rank']=np.where(df_3a4.REVENUE_NON_REVENUE=='YES', 0, 1)

    ##### Step3: sort the SS per ranking columns and Put MFG hold orders at the back
    df_3a4.sort_values(by=ranking_col, ascending=True, inplace=True)
    # Put MFG hold orders at the back
    df_hold=df_3a4[df_3a4.MFG_HOLD=='Y'].copy()
    df_3a4=df_3a4[df_3a4.MFG_HOLD!='Y'].copy()
    df_3a4=pd.concat([df_3a4,df_hold],sort=False)

    ##### Step3: create rank# and put in 3a4
    rank = {}
    order_list = df_3a4[order_col].unique()
    for order, rk in zip(order_list, range(1, len(order_list) + 1)):
        rank[order] = rk
    df_3a4.loc[:, new_col] = df_3a4[order_col].map(lambda x: rank[x])

    return df_3a4


- Read 3a4, OH, Intransit, scr
- Create Oh dict
- Create intransit dict - how to deal with ETA TBD
- Process 3a4
- 3A4 consume DF OH by site
- 3A4 consume intransit - how to deal with ETA TBD
- Rank 3A4
- Create scr dict
- Allocate scr to 3a4
- Summarize allocation result


In [551]:
# read 3a4
df_3a4=pd.read_csv(f_3a4, encoding='ISO-8859-1',parse_dates=['CURRENT_FCD_NBD_DATE',  'ORIGINAL_FCD_NBD_DATE'],low_memory=False)


In [552]:
# read scr
df_scr=pd.read_excel(f_supply,sheet_name=sheet_scr)

In [553]:
# read oh this includes PCBA SM, will be removed when creating DF OH dict
df_oh=pd.read_excel(f_supply,sheet_name=sheet_oh)


In [554]:
# read in-transit
df_transit=pd.read_excel(f_supply,sheet_name=sheet_transit)

In [555]:
# pivot df_oh
df_oh=df_oh.pivot_table(index=['planningOrg','TAN'],values='OH',aggfunc=sum)

In [556]:
# versionless df_oh
df_oh=change_supply_to_versionless_and_addup_supply(df_oh,pn_col='TAN')

In [557]:
# 生成OH dict；
oh_dic_tan=created_oh_dict_per_df_oh(df_oh,pcba_site)

In [558]:
# extract BU info for TAN from SCR
tan_bu=extract_bu_from_scr(df_scr)

In [559]:
# Pivot df_scr 并处理日期格式
df_scr=df_scr.pivot_table(index=['planningOrg','TAN'],columns='SCRDate',values='SCRQuantity',aggfunc=sum)
df_scr.columns=df_scr.columns.map(lambda x: x.date())


In [560]:
# versionless df_scr
df_scr=change_supply_to_versionless_and_addup_supply(df_scr,pn_col='TAN')

In [561]:
# simplify the index will make it much faster to get the dict
df_scr.reset_index(inplace=True)
df_scr.drop('planningOrg',axis=1,inplace=True)
df_scr.set_index('TAN',inplace=True)
supply_dic_tan=created_supply_dict_per_scr(df_scr)


In [562]:
# Offset 3A4 OSSD and FCD by transit time
df_3a4.loc[:,'fcd_offset']=df_3a4.apply(lambda x: update_date_with_transit_pad(x.ORGANIZATION_CODE, x.CURRENT_FCD_NBD_DATE,transit_time,pcba_site),axis=1)
df_3a4.loc[:,'ossd_offset']=df_3a4.apply(lambda x: update_date_with_transit_pad(x.ORGANIZATION_CODE, x.ORIGINAL_FCD_NBD_DATE,transit_time,pcba_site),axis=1)


In [563]:
# Rank the orders
df_3a4=ss_ranking_overall_new(df_3a4,ranking_col, order_col='SO_SS', new_col='ss_overall_rank')


In [564]:
# (do below after ranking) Process 3a4 BOM base on FLB_TAN col
df_bom = generate_df_order_bom_from_flb_tan_col(df_3a4,supply_dic_tan.keys())
df_3a4 = update_order_bom_to_3a4(df_3a4, df_bom)

In [565]:
# create backlog dict for Tan exists in SCR
blg_dic_tan=create_blg_dict_per_sorted_3a4_and_selected_tan(df_3a4,supply_dic_tan.keys())


In [566]:
# Oh to fulfill backlog per site. update blg_dic_tan accordingly
blg_dic_tan=fulfill_backlog_by_oh(oh_dic_tan, blg_dic_tan)

In [567]:
# TODO: 按照org将in-transit分配给自己的订单（TBD: consider ETA or not - if new supply is considered to be availabe later than any intransit, then no need consider ETA but just total qty）
# - 更新blg_dic_tan


In [568]:
# Allocate SCR and 生成allocated supply dict
supply_dic_tan_allocated=allocate_supply_per_supply_and_blg_dic(supply_dic_tan,blg_dic_tan)


In [569]:
#生成聚合的allocated supply dict
supply_dic_tan_allocated_agg=aggregate_supply_dic_tan_allocated(supply_dic_tan_allocated)


In [570]:
# 在df_scr中加入allocation结果
df_scr=add_allocation_to_scr(df_scr,df_3a4,supply_dic_tan_allocated_agg,pcba_site)

In [571]:
#把以下信息加回scr: BU, backlog, OH, intransit; 并做相应的计算处理
df_scr=process_final_allocated_output(df_scr,tan_bu,df_3a4,df_oh,pcba_site)

                  BU  Backlog     OH  Gap_before  Allocation  Gap_after  \
TAN       ORG                                                             
68-100463 FJZ   UABU      1.0    NaN        -1.0         1.0        0.0   
          SHK   UABU    247.0  163.0       -84.0        84.0        0.0   
          SJZ   UABU    657.0    NaN      -657.0       657.0        0.0   
68-100465 SJZ   UABU     70.0    NaN       -70.0        70.0        0.0   
68-100466 SHK   UABU    231.0   46.0      -185.0       185.0        0.0   
...              ...      ...    ...         ...         ...        ...   
800-48419 FGU  SRGBU      4.0    NaN        -4.0         4.0        0.0   
          FOC  SRGBU      7.0    5.0        -2.0         2.0        0.0   
800-48768 FOC  INSBU     29.0   17.0       -12.0        12.0        0.0   
800-48960 FTX  CSPBU     29.0    0.0       -29.0        29.0        0.0   
800-49119 FOC  INSBU      8.0    3.0        -5.0         5.0        0.0   

              Recovery 2

In [572]:
df_scr.to_excel('scr with allocation.xlsx')

In [573]:
df_scr.reset_index(inplace=True)
summary=df_scr.pivot_table(index='TAN',values='ORG',aggfunc=len)
summary[summary.ORG>1]

,ORG
TAN,
68-100463,5
68-100465,4
68-100466,4
68-100467,5
68-100468,4
...,...
800-49638,2
800-49639,2
800-49759,3
